# Assignment 1
## Hossein Soltanloo (810195407) 

In [1]:
import copy
import time
from collections import deque

UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

X = 0
Y = 1
CAP = 2


In [66]:
def twoDimensionalListToTuple(snapshot):
    listOfTuples = []
    for row in snapshot:
        listOfTuples.append(tuple(row))
    return tuple(listOfTuples)

def tupleReplaceIndex(tup, ix, val):
    return tup[:ix] + (val,) + tup[ix+1:]

class MapSnapshot:
    def __init__(self, _ambulanceLocation, _mapInfo, _hospitalsInfo, _move):
        self.ambulanceLocation = _ambulanceLocation
        self.mapInfo = _mapInfo
        self.hospitalsInfo = _hospitalsInfo
        self.inputMove = _move
        
    def generateNext(self, move):
        if move == UP:
#             if self.inputMove==DOWN:
#                 return 0, None
            nextLoc = (self.ambulanceLocation[X], self.ambulanceLocation[Y] - 1)
            doubleNextLoc = (self.ambulanceLocation[X], self.ambulanceLocation[Y] - 2)
            
        if move == DOWN:
#             if self.inputMove==UP:
#                 return 0, None
            nextLoc = (self.ambulanceLocation[X], self.ambulanceLocation[Y] + 1)
            doubleNextLoc = (self.ambulanceLocation[X], self.ambulanceLocation[Y] + 2)
            
        if move == RIGHT:
#             if self.inputMove==LEFT:
#                 return 0, None
            nextLoc = (self.ambulanceLocation[X] + 1, self.ambulanceLocation[Y])
            doubleNextLoc = (self.ambulanceLocation[X] + 2, self.ambulanceLocation[Y])
            
        if move == LEFT:
#             if self.inputMove==RIGHT:
#                 return 0, None
            nextLoc = (self.ambulanceLocation[X] - 1, self.ambulanceLocation[Y])
            doubleNextLoc = (self.ambulanceLocation[X] - 2, self.ambulanceLocation[Y])
            
        if self.mapInfo[nextLoc[Y]][nextLoc[X]] == '#' or (self.mapInfo[nextLoc[Y]][nextLoc[X]] == 'P' and (self.mapInfo[doubleNextLoc[Y]][doubleNextLoc[X]] == '#' or self.mapInfo[doubleNextLoc[Y]][doubleNextLoc[X]] == 'P')):
            return 0,None
        elif self.mapInfo[nextLoc[Y]][nextLoc[X]] == ' ' or (self.mapInfo[nextLoc[Y]][nextLoc[X]]).isdigit():
            newHospitals = dict(self.hospitalsInfo)
            newMap = list(self.mapInfo)
            newMap[nextLoc[Y]] = tupleReplaceIndex(newMap[nextLoc[Y]], nextLoc[X], 'A')
            newMap[self.ambulanceLocation[Y]] = tupleReplaceIndex(newMap[self.ambulanceLocation[Y]], self.ambulanceLocation[X], ' ')
            if self.ambulanceLocation in newHospitals:
                newMap[self.ambulanceLocation[Y]] = tupleReplaceIndex(newMap[self.ambulanceLocation[Y]], self.ambulanceLocation[X], str(newHospitals[self.ambulanceLocation]))
            return 1,MapSnapshot(nextLoc, newMap, newHospitals,move)
        elif self.mapInfo[nextLoc[Y]][nextLoc[X]] == 'P':
            if self.mapInfo[doubleNextLoc[Y]][doubleNextLoc[X]] == ' ':
                newHospitals = dict(self.hospitalsInfo)
                newMap = list(self.mapInfo)
                newMap[nextLoc[Y]] = tupleReplaceIndex(newMap[nextLoc[Y]], nextLoc[X], 'A')
                newMap[doubleNextLoc[Y]] = tupleReplaceIndex(newMap[doubleNextLoc[Y]], doubleNextLoc[X], 'P')
                newMap[self.ambulanceLocation[Y]] = tupleReplaceIndex(newMap[self.ambulanceLocation[Y]], self.ambulanceLocation[X], ' ')
                if self.ambulanceLocation in newHospitals:
                    newMap[self.ambulanceLocation[Y]] = tupleReplaceIndex(newMap[self.ambulanceLocation[Y]], self.ambulanceLocation[X], str(newHospitals[self.ambulanceLocation]))
                return 1,MapSnapshot(nextLoc, newMap, newHospitals,move)
            else:
                newHospitals = dict(self.hospitalsInfo)
                newMap = list(self.mapInfo)
                newMap[nextLoc[Y]] = tupleReplaceIndex(newMap[nextLoc[Y]], nextLoc[X], 'A')
                newMap[doubleNextLoc[Y]] = tupleReplaceIndex(newMap[doubleNextLoc[Y]], doubleNextLoc[X], str(int(newMap[doubleNextLoc[Y]][doubleNextLoc[X]]) - 1))
                newHospitals[doubleNextLoc] -= 1
                newMap[self.ambulanceLocation[Y]] = tupleReplaceIndex(newMap[self.ambulanceLocation[Y]], self.ambulanceLocation[X], ' ')
                if int(newMap[doubleNextLoc[Y]][doubleNextLoc[X]]) == 0:
                    newMap[doubleNextLoc[Y]] = tupleReplaceIndex(newMap[doubleNextLoc[Y]], doubleNextLoc[X], ' ')
                    del newHospitals[doubleNextLoc]
                if self.ambulanceLocation in newHospitals:
                    newMap[self.ambulanceLocation[Y]] = tupleReplaceIndex(newMap[self.ambulanceLocation[Y]], self.ambulanceLocation[X], str(newHospitals[self.ambulanceLocation]))
                return 1, MapSnapshot(nextLoc, newMap, newHospitals, move)
        
    def hasNoPatients(self):
        hasPatients = False
        for row in self.mapInfo:
            if 'P' in row:
                hasPatients = True
        return not hasPatients
    
    def __eq__(self, other):
        if self.mapInfo == other.mapInfo and self.hospitalsInfo == other.hospitalsInfo:
            return True
        
    def __hash__(self):
        return (hash(tuple(self.mapInfo)))
        
    def __str__(self):
        res = ""
        for row in self.mapInfo:
            res += "".join(row)
            res += "\n"
        return(res)
        
    @staticmethod
    def parseMap(filepath):
        mapInfo = []
        ambulance = ()
        hospitalsInfo = {}
        with open(filepath) as fp:
            for rowNo, line in enumerate(fp):
                lineList = []
                for colNo, c in enumerate(line):
                    if c == '\n':
                        continue
                    lineList.append(c)
                    if c == 'A':
                        ambulance = (colNo, rowNo)
                    if c.isdigit():
                        hospitalsInfo[(colNo, rowNo)] = int(c)
                mapInfo.append(tuple(lineList))
        return MapSnapshot(ambulance, mapInfo, hospitalsInfo, -1)
                
class State:
    def __init__(self, _parent, _snapshot, _height):
        self.parent = _parent
        self.snapshot = _snapshot
        self.children = []
        self.height = _height
        
    def getHeight(self):
        return self.height
        
    def generateChildren(self):
        for i in range(0, 4):
            state, nextchild = self.snapshot.generateNext(i)
            if(state==1):
                child = State(self, nextchild, self.height + 1)
                self.children.append(child)
    
    def getChildren(self):
        return self.children
    
    def getSnapshot(self):
        return self.snapshot
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.snapshot == other.snapshot
        else:
            return False
        
    def isGoalState(self):
        return self.snapshot.hasNoPatients()
    
    def __hash__(self):
        return hash(self.snapshot)
    
    def __str__(self):
        return str(self.snapshot)

In [67]:
testFileNames = ['test1', 'test2', 'test3']

In [68]:
def breadthFirstSearch(startState):
    visitedStates = 0
    visitedUniqueStates = 0
    pathCost = 0
    if startState.isGoalState():
        return startState
    frontier = deque()
    frontier.append(startState)
    explored = set()
    
    while True:
        if not frontier:
            return None
        node = frontier.popleft()
        visitedStates += 1

        if node not in explored:
            visitedUniqueStates += 1
            explored.add(node)
            if node.isGoalState():
                return (node, visitedStates, visitedUniqueStates, node.getHeight())
            node.generateChildren()
            children = node.getChildren()
            for child in children:
                if child not in explored:
                    frontier.append(child)

In [69]:
def calculateSteps(node):
    solutionSteps = 0
    print(node)
    while(True):
        node = node.parent
        if(node!=None):
            solutionSteps+=1
            print(node)
        else:
            break
    
    return solutionSteps


startState = State(None, MapSnapshot.parseMap('testcases/test1.txt'), 0)
start = time.time()
goal = breadthFirstSearch(startState)
end = time.time()

calculateSteps(goal[0])

print((end - start))
print("Visited States: " + str(goal[1]))
print("Visited Unique States: " + str(goal[2]))
print(goal[3])

######
#    #
# #  #
#    #
#A   #
#    #
######

######
#    #
# #  #
#A   #
#P   #
#1   #
######

######
#    #
# #  #
# A  #
#P   #
#1   #
######

######
#    #
# #  #
#    #
#PA  #
#1   #
######

######
#    #
# #  #
#    #
#P   #
#1A  #
######

######
#    #
# #  #
#    #
#P   #
#2PA #
######

######
#    #
# #  #
#    #
#P   #
#2 PA#
######

######
#    #
# #  #
#    #
#P  A#
#2 P #
######

######
#    #
# #  #
#    #
#P A #
#2 P #
######

######
#    #
# #  #
#  A #
#P P #
#2   #
######

######
#    #
# #A #
#  P #
#P   #
#2   #
######

######
#  A #
# #P #
#    #
#P   #
#2   #
######

0.010577917098999023
Visited States: 519
Visited Unique States: 415
11


In [70]:
# def depthLimitedSearch(startState, depth):
#     if(depth == 0):
#         if(not startState.isGoalState()):
#             return False, None
#         else:
#             return True, startState
#     if startState.isGoalState():
#         return True, startState
    
#     if(depth == 0):
#         return False, None

#     if not startState.getChildren():
#         startState.generateChildren()

#     children = startState.getChildren()
    
#     for child in children:
#         found, state = dls(child, depth - 1)
#         if(found):
#             return found, state

#     return False, None

def dls(startState, maxDepth):
    print(maxDepth)
    visited = 0
    frontier = [startState]
    explored = [set() for i in range(maxDepth+1)]
    gexplored = set()
    while frontier:
        node = frontier.pop()
#         print(node.getHeight())
        gexplored.add(node)
#         for i in range(node.getHeight(), maxDepth):
#             explored[i].add(node)
        visited+=1
        if node.isGoalState():
            return (True, node, visited)
        if node.getHeight() < maxDepth - 1:
            node.generateChildren()
            for child in node.getChildren()[::-1]:
#                 if child not in explored[node.getHeight()]:
                if child not in gexplored:
                    frontier.append(child)

    return (False, None, visited)

# def searchToDepth(startState, depth):
#     visited = 0
#     explored = set()
#     if(depth == 0):
#         if(startState.isGoalState()):
#             return True, startState, 0
#         else:
#             return False, startState, 0

#     if(startState.getChildren() == []):
#         startState.generateChildren()

#     expansion = startState.getChildren()
#     for child in expansion:
#         visited += 1
#         if(child not in explored):
#             found, state,v = searchToDepth(child, depth-1)
#             explored.add(child)
#             visited += v
#             if(found):
#                 return found, state, visited

#     return False, startState, visited


def iterativeDeepeningSearch(startState):
    limit = 0
    while True:
        found, state, visited = dls(startState, limit)
        print(visited)
        if(found):
            return state
        limit += 1
    return None

In [71]:
startState = State(None, MapSnapshot.parseMap('testcases/test1.txt'), 0)
print("here1")


here1


In [72]:
start = time.time()
print("here2")


here2


In [73]:
goal = iterativeDeepeningSearch(startState)
end = time.time()
print(calculateSteps(goal))

print((end - start))

0
1
1
1
2
4
3
12
4
29
5
61
6
107
7
173
8
268
9
361
10
507
11
569
12
672
13
806
14
894
15
1040
16
1306
17
1624
18
2072
19
2061
######
#    #
# #  #
#    #
#A   #
#    #
######

######
#    #
# #  #
#A   #
#P   #
#1   #
######

######
#    #
# #  #
# A  #
#P   #
#1   #
######

######
#    #
# #  #
#    #
#PA  #
#1   #
######

######
#    #
# #  #
#    #
#P   #
#1A  #
######

######
#    #
# #  #
#    #
#P   #
#2PA #
######

######
#    #
# #  #
#    #
#P   #
#2 PA#
######

######
#    #
# #  #
#    #
#P  A#
#2 P #
######

######
#    #
# #  #
#   A#
#P   #
#2 P #
######

######
#    #
# # A#
#    #
#P   #
#2 P #
######

######
#   A#
# #  #
#    #
#P   #
#2 P #
######

######
#  A #
# #  #
#    #
#P   #
#2 P #
######

######
#    #
# #A #
#    #
#P   #
#2 P #
######

######
#    #
# #  #
#  A #
#P   #
#2 P #
######

######
#    #
# #  #
#    #
#P A #
#2 P #
######

######
#    #
# #  #
#  A #
#P P #
#2   #
######

######
#    #
# #A #
#  P #
#P   #
#2   #
######

######
#  A #
# #P #
#  

In [ ]:
import queue

def aStar(startState):
    startState.generateChildren()
    
    if startState.isGoalState():
        return start

    expansion = queue.PriorityQueue()
    goals = queue.PriorityQueue()
    list(map(expansion.put, startState.getChildren()))

    while expansion:
        node = expansion.get()

        if(node.hasConflict() == False):
            if(goals.empty()):
                goals.put(node)
            else:
                g = goals.get()
                if(g.priority < node.priority):
                    return g
                else:
                    goals.put(node)

        node.generateChildren(node.height+1, prune=0)

        list(map(expansion.put, node.getChildren()))
    return None